https://keras.io/guides/keras_nlp/getting_started/

In [1]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# Use mixed precision to speed up all training in this guide.
keras.mixed_precision.set_global_policy("mixed_float16")

2024-07-23 02:40:49.062373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 02:40:49.062489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 02:40:49.267054: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  2491k      0  0:00:32  0:00:32 --:--:-- 2664k


In [4]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

# Inspect first review
# Format is (review text tensor, label tensor)
print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'We went into this movie because my husband had enjoyed the original version of `My favourite Martian\'. We had our 6 year old daughter with us. She wanted to leave halfway through the movie which was fine with both her parents! The parts we did see were only occasionally humorous, mostly either too silly or gross. I would expect that this movie might appeal to kids between 9-12, if that. It\'s definitely not suited for younger children. From what I\'ve heard the original series was by far superior and if you are going to "relive the past" you\'ll probably be disappointed.'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


In [5]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

Attaching 'model.safetensors' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'task.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 84 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_tiny_en_uncased_sst2/4' to your Kaggle notebook...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[-1.287,  1.393]], dtype=float16)

In [ ]:
classifier.evaluate(imdb_test)

 194/1563 ━━━━━━━━━━━━━━━━━━━━ 10:27 458ms/step - loss: 0.4493 - sparse_categorical_accuracy: 0.7960